# Energy Innovation MCOE Compilation

#### Contents
- Setup
- Part 1: Basic Plant & Unit Information
- Part 2: Cost Data
- Part 3: Emissions & Public Health Data

## Setup

In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import pudl
import pandas as pd
import sqlalchemy as sa
import sys
import logging
import pathlib
import json

In [3]:
# basic setup for logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]
pd.options.display.max_columns = None

In [4]:
# pudl_settings is a dictionary that includes the paths to several key pudl directories
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])

In [5]:
# the creation of the pudl_out object
# this will compile output tables like mcoe - if you want to restart/wipe the saved elements rerun this cell
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine, freq='AS', rolling=True)

In [6]:
# run mcoe method... the original function is in pudl.analysis.mcoe.mcoe()
mcoe = pudl_out.mcoe()

filling in fuel cost NaNs with rolling averages


----------
## Part 1: Basic Plant & Unit Information

#### Plant Level Data:  <font color=blue>*plant_level_df* </font><font color=#459cff>*(plant_level_df_all_years)*</font>
Subdivided by broad fuel type (coal, gas, oil, waste) by most recent reporting date. See plant_level_df_all_years for all years.

In [7]:
# Make subset of mcoe data with desired columns
mcoe_small = mcoe[['plant_id_pudl',
                   'report_date',
                   'fuel_type_code_pudl',
                   'plant_name_eia',
                   'state',
                   'city',
                   'latitude',
                   'longitude']].drop_duplicates()

In [8]:
# Group by pudl id and broad fuel type (all years)
plant_fuel_group_all_years = mcoe_small.groupby([
    'plant_id_pudl',
    'fuel_type_code_pudl',
    'report_date'],as_index=False).first()

# Group by pudl id and broad fuel type and select most recent date
plant_fuel_group = mcoe_small.groupby([
    'plant_id_pudl',
    'fuel_type_code_pudl'],as_index=False)['report_date'].max()

# Merge with rest of the dataframe fields
plant_level_df = pd.merge(plant_fuel_group,mcoe_small,on=['plant_id_pudl','fuel_type_code_pudl','report_date'],how='inner').sort_values('plant_id_pudl')
plant_level_df_all_years = pd.merge(plant_fuel_group_all_years,mcoe_small,on=['plant_id_pudl','fuel_type_code_pudl','report_date'],how='inner')

In [9]:
plant_level_df

,plant_id_pudl,fuel_type_code_pudl,report_date,plant_name_eia,state,city,latitude,longitude
0,1,coal,2018-01-01,Dolet Hills,LA,Mansfield,32.030556,-93.569167
1,2,coal,2018-01-01,Flint Creek,AR,Gentry,36.256100,-94.524100
2,3,coal,2018-01-01,Pirkey,TX,Hallsville,32.460700,-94.485200
3,6,coal,2018-01-01,Allen S King,MN,Oak Park Heights,45.030000,-92.778600
4,8,coal,2018-01-01,A B Brown,IN,Mount Vernon,37.905300,-87.715000
...,...,...,...,...,...,...,...,...
621,8297,coal,2013-01-01,Fair Station,IA,Muscatine,41.456900,-90.823300
622,8299,gas,2011-01-01,Harvey Couch,AR,Stamps,33.359200,-93.464900
623,8446,gas,2018-01-01,Brunswick County Power Station,VA,Freeman,36.765000,-77.713889
624,9938,gas,2018-01-01,Carty Generating Station,OR,Boardman,45.698611,-119.813056


#### Unit Level Data: <font color=blue>*unit_level_df* </font><font color=#459cff>*(unit_level_df_all_years)*</font>
Subdivided by plant, unit, and broad fuel type by most recent reporting date. Generator data aggregated to unit level. See unit_level_df_all_years for all years.

In [10]:
mcoe_gen_specific = mcoe[[
    'plant_id_pudl',
    'unit_id_pudl',
    'generator_id',
    'fuel_type_code_pudl',
    'report_date',
    'nameplate_power_factor',
    'generator_age_years',
    #'planned_retirement_date',
    #'total_mmbtu',
    'capacity_mw',
    'net_generation_mwh',
    'heat_rate_mmbtu_mwh',
]].drop_duplicates()

mcoe_unit_specific = mcoe[[
    'plant_id_pudl',
    'unit_id_pudl',
    'fuel_type_code_pudl',
    'report_date',
    'technology_description',
    'state',
    'city',
    'latitude',
    'longitude',
]].drop_duplicates()

In [11]:
# Function to calculate weighted average
def weighted_average(df, data_col, weight_col, by_col, new_col_name):
    """Generate a weighted average."""
    df['_data_times_weight'] = df[data_col] * df[weight_col]
    df['_weight_where_notnull'] = df[weight_col] * pd.notnull(df[data_col])
    g = df.groupby(by_col)
    result = g['_data_times_weight'].sum() / g['_weight_where_notnull'].sum()
    del df['_data_times_weight'], df['_weight_where_notnull']
    return result.to_frame(name=new_col_name).reset_index()

In [12]:
# Find weighted average of generator ages (based on capacity) and heat rates (based on net gen) per unit
gen_age_wAve_df = weighted_average(mcoe_gen_specific,'generator_age_years','capacity_mw',['plant_id_pudl','unit_id_pudl','fuel_type_code_pudl','report_date'],'weighted_ave_gen_age_years')
heat_rate_wAve_df = weighted_average(mcoe_gen_specific,'heat_rate_mmbtu_mwh','net_generation_mwh',['plant_id_pudl','unit_id_pudl','fuel_type_code_pudl','report_date'],'weighted_ave_heat_rate_mwh')

# Find sum of other relevant fields per unit
unit_sum_df = mcoe_gen_specific.groupby([
    'plant_id_pudl',
    'unit_id_pudl',
    'fuel_type_code_pudl',
    'report_date'],as_index=False)[[#'total_mmbtu',
                    'capacity_mw',
                    'net_generation_mwh',
                    'nameplate_power_factor']].sum()

In [13]:
# Merge unit conglomerates into one table
age_heat_rate_merge = pd.merge(gen_age_wAve_df, heat_rate_wAve_df, on=['plant_id_pudl','unit_id_pudl','fuel_type_code_pudl','report_date'],how='outer')
wAve_sum_merge = pd.merge(age_heat_rate_merge, unit_sum_df, on=['plant_id_pudl','unit_id_pudl','fuel_type_code_pudl','report_date'],how='outer')

# For some reason this merge incurs duplicates WHY? (see drop_duplicates)
unit_level_df_all_years = pd.merge(wAve_sum_merge, mcoe_unit_specific, on=['plant_id_pudl',
                                                                           'unit_id_pudl',
                                                                           'fuel_type_code_pudl',
                                                                           'report_date'],how='left').drop_duplicates()

In [14]:
# Separate by most recent reporting date
unit_level_df = unit_level_df_all_years.groupby([
    'plant_id_pudl',
    'unit_id_pudl',
    'fuel_type_code_pudl'],as_index=False)[['report_date']].max()

# Merge back with full data
unit_level_df = pd.merge(unit_level_df,unit_level_df_all_years,on=['plant_id_pudl','unit_id_pudl','fuel_type_code_pudl','report_date'],how='left')

In [15]:
# --- SANITY CHECK ---
#import datetime
#unit_level_df.loc[unit_level_df['plant_id_pudl']==32]
#unit_level_df_all_years.loc[(unit_level_df_all_years['plant_id_pudl']==32) & (unit_level_df_all_years['report_date']==datetime.datetime(2017,1,1))]

In [16]:
unit_level_df

,plant_id_pudl,unit_id_pudl,fuel_type_code_pudl,report_date,weighted_ave_gen_age_years,weighted_ave_heat_rate_mwh,capacity_mw,net_generation_mwh,nameplate_power_factor,technology_description,state,city,latitude,longitude
0,1,1,coal,2018-01-01,34.0,12.371128,720.7,1483069.0,0.93,Conventional Steam Coal,LA,Mansfield,32.030556,-93.569167
1,2,1,coal,2018-01-01,42.0,10.614269,558.0,2669748.0,0.90,Conventional Steam Coal,AR,Gentry,36.256100,-94.524100
2,3,1,coal,2018-01-01,35.0,10.423375,721.0,4585248.0,0.90,Conventional Steam Coal,TX,Hallsville,32.460700,-94.485200
3,6,1,coal,2018-01-01,62.0,9.808772,598.4,2698691.0,0.85,Conventional Steam Coal,MN,Oak Park Heights,45.030000,-92.778600
4,8,1,coal,2018-01-01,41.0,11.267235,265.2,1183700.0,0.90,Conventional Steam Coal,IN,Mount Vernon,37.905300,-87.715000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,8297,2,coal,2013-01-01,53.0,12.102598,37.5,175926.0,0.85,None,IA,Muscatine,41.456900,-90.823300
1313,8299,1,gas,2011-01-01,66.0,14.834102,156.2,21817.0,0.00,None,AR,Stamps,33.359200,-93.464900
1314,8446,1,gas,2018-01-01,4.0,NaN,1174.7,0.0,2.55,Natural Gas Fired Combined Cycle,VA,Freeman,36.765000,-77.713889
1315,9938,1,gas,2018-01-01,4.0,6.851713,500.0,3020826.0,1.80,Natural Gas Fired Combined Cycle,OR,Boardman,45.698611,-119.813056


----------------
## Part 2: Cost Data

#### Plant Level Data: <font color=blue>*mcoe_plant_df*</font>
Subdivided by plant and broad fuel type.


MCOE Variables & Origins:
- Fuel cost = **EIA**: *total_fuel_cost*
- Variable O&M = **FERC**: *(opex_production_total) - (opex_fuel)*
- Fixed O&M = **FERC**: *capex_total*
- MW Capacity = **FERC**: *capacity_mw*
- Net MWh Generated = **EIA**: *net_generation_mwh*

##### **EIA Data**: <font color=grey>*eia_plant_fuel_ag*</font>
Grab pertinant EIA data and group by plant id, fuel, and year so that it's ready to merge with ferc later (this df implicated in grey above). Next, find the percent of net generation and capacity attributable to each fuel type per plant. These percent values will be used to disagregate FERC fixed and variable cost data by fuel type.


In [17]:
# Grab relevant EIA plant fields from mcoe table
mcoe_relevant = mcoe[[
    'report_date',
    'plant_id_pudl',
    'unit_id_pudl',
    'generator_id',
    'fuel_type_code_pudl',
    'total_fuel_cost',
    'net_generation_mwh',
    'capacity_mw'
]]#.drop_duplicates() #but why are there so many??? 

In [18]:
# Sum unit-level cost and generation fields by plant and fuel type
eia_plant_fuel_ag = mcoe_relevant.groupby(
    ['plant_id_pudl',
     'fuel_type_code_pudl',
     'report_date'],as_index=False)[['total_fuel_cost',
                                     'net_generation_mwh',
                                     'capacity_mw']].sum()

# Change report_date to report_year to ensure compatability with FERC.
# This will be the database that is merged with FERC later on.
eia_plant_fuel_ag['report_year'] = eia_plant_fuel_ag['report_date'].apply(lambda x: x.year)
eia_plant_fuel_ag = eia_plant_fuel_ag.drop(['report_date'],axis=1)

In [19]:
# --- SANITY CHECK --- 
#len(eia_plant_fuel_ag)
#eia_plant_fuel_ag.loc[eia_plant_fuel_ag['plant_id_pudl']==32].sort_values('report_year')
#mcoe_relevant[mcoe_relevant.duplicated()]

In [20]:
# Calculate plant totals for net generation and capacity
only_plants = eia_plant_fuel_ag.groupby(['plant_id_pudl','report_year']).sum()
only_plants = only_plants.rename(columns={'net_generation_mwh':'total_net_gen_mwh','capacity_mw':'total_capacity_mw'})
only_plants = only_plants.drop(['total_fuel_cost'],axis=1)

# Add plant totals back to full eia_plant_fuel_ag dataframe
totals_merge = pd.merge(eia_plant_fuel_ag, only_plants, on=['plant_id_pudl','report_year'],how='right')

# Rename fuel field to avoid confusion for what 'total' means in this case
totals_merge = totals_merge.rename(columns={'total_fuel_cost':'fuel_cost'})

# Calculate coal and gas portion's percent of plant's net generation and capacity.
totals_merge['%_net_gen'] = totals_merge['net_generation_mwh'] / totals_merge['total_net_gen_mwh']
totals_merge['%_capacity'] = totals_merge['capacity_mw'] / totals_merge['total_capacity_mw']

In [21]:
# --- SANITY CHECK --- that total is the same for same year different fuel type
#len(totals_merge)
#totals_merge.loc[(totals_merge['plant_id_pudl']==32)].sort_values(['report_year'],ascending=False)

In [22]:
# Reorient tables so that calculated % values for fuel types become columns vs. rows.
# Rename to specify which percent they are refering to.
pct_net_gen = totals_merge.pivot_table('%_net_gen',['plant_id_pudl','report_year'],'fuel_type_code_pudl').reset_index()
pct_net_gen = pct_net_gen.rename(columns={'coal':'%_net_gen_coal',
                                          'gas':'%_net_gen_gas',
                                          'oil':'%_net_gen_oil',
                                          'waste':'%_net_gen_waste'})
pct_cap = totals_merge.pivot_table('%_capacity',['plant_id_pudl','report_year'],'fuel_type_code_pudl').reset_index()
pct_cap = pct_cap.rename(columns={'coal':'%_cap_coal',
                                  'gas':'%_cap_gas',
                                  'oil':'%_cap_oil',
                                  'waste':'%_cap_waste'})

In [23]:
# Merge % net gen and % cap tables 
eia_pct_merge = pd.merge(pct_net_gen, pct_cap, on=['plant_id_pudl','report_year'],how='outer')

In [24]:
# --- SANITY CHECK --- that total is the same for same year different fuel type
#len(eia_pct_merge)
#len(totals_merge)
#totals_merge.loc[(totals_merge['plant_id_pudl']==32)].sort_values(['report_year'],ascending=False)
#eia_pct_merge.loc[(eia_pct_merge['plant_id_pudl']==32)].sort_values(['report_year'],ascending=False)

##### **FERC Form 1 Data:** <font color=grey>*ferc1_steam_merge*</font>
Grab relevant FERC data and group by plant and report date. Prepare for disaggregation based on EIA fuel percentage calculations.

In [25]:
# Grab relevant FERC1 steam plant fields
ferc1_steam = pudl_out.plants_steam_ferc1()[[
    'report_year',
    'plant_id_pudl',
    'capex_total',
    'opex_fuel',
    'opex_production_total',
]] #can't drop dublicates because entries for unique generators under same plant id

In [26]:
# Count number of entries under a single pudl id and year
ferc1_steam_count = ferc1_steam.groupby(
    ['plant_id_pudl','report_year']).size().reset_index(name='count')

# Sum other fields together so there is one plant value per year
ferc1_steam_sum = ferc1_steam.groupby(
    ['plant_id_pudl','report_year'],as_index=False).sum()

# Merge based on year and pudl id so that count is incorporated
ferc1_steam_merge = pd.merge(ferc1_steam_count,ferc1_steam_sum,on=['plant_id_pudl','report_year'])

# Calculate variable O&M
ferc1_steam_merge['opex_nofuel_ferc1'] = ferc1_steam_merge['opex_production_total']-ferc1_steam_merge['opex_fuel']

# Rename report_year to specify its connection to FERC data
ferc1_steam_merge = ferc1_steam_merge.rename(columns={'count':'count_ferc1',
                                                      'capex_total':'capex_total_ferc1',
                                                      'opex_fuel':'opex_fuel_ferc1',
                                                      'opex_production_total':'opex_production_total_ferc1'})

In [27]:
# --- SANITY CHECK --- to see if 'count' field is correct
#ferc1_steam.loc[ferc1_steam['plant_id_pudl']==32].sort_values('report_year',ascending=False)
#ferc1_steam_merge.loc[(ferc1_steam_merge['plant_id_pudl']==32) & (ferc1_steam_merge['report_year']==2018)]

##### **FERC & EIA Merge #1:** <font color=grey>*ferc_cap_op*</font>
Disaggregate FERC data by fuel type using EIA percentage breakdowns; calculate varable (opex) and fixed (capex) costs for FERC plant data by fuel type.

In [28]:
# Merge FERC data with EIA fuel percentange breakdowns.

# ---- HOW SHOULD WE MERGE? RN HAVE AS OUTER, BUT MAYBE INNER OR LEFT IS BETTER?) ----

ferc_eia_pcts = pd.merge(eia_pct_merge,ferc1_steam_merge,on=['plant_id_pudl','report_year'],how='outer')

In [29]:
# --- SANITY CHECK --- 
#len(ferc_eia_pcts)
#ferc_eia_pcts.loc[ferc_eia_pcts['plant_id_pudl']==32]

In [30]:
# Create columns for total costs broken down by fuel type
ferc_eia_pcts['capex_coal'] = ferc_eia_pcts['capex_total_ferc1']*ferc_eia_pcts['%_cap_coal']
ferc_eia_pcts['capex_gas'] = ferc_eia_pcts['capex_total_ferc1']*ferc_eia_pcts['%_cap_gas'] 
ferc_eia_pcts['capex_oil'] = ferc_eia_pcts['capex_total_ferc1']*ferc_eia_pcts['%_cap_oil']
ferc_eia_pcts['capex_waste'] = ferc_eia_pcts['capex_total_ferc1']*ferc_eia_pcts['%_cap_waste']

ferc_eia_pcts['opex_coal'] = ferc_eia_pcts['opex_nofuel_ferc1']*ferc_eia_pcts['%_net_gen_coal']
ferc_eia_pcts['opex_gas'] = ferc_eia_pcts['opex_nofuel_ferc1']*ferc_eia_pcts['%_net_gen_gas']
ferc_eia_pcts['opex_oil'] = ferc_eia_pcts['opex_nofuel_ferc1']*ferc_eia_pcts['%_net_gen_oil']
ferc_eia_pcts['opex_waste'] = ferc_eia_pcts['opex_nofuel_ferc1']*ferc_eia_pcts['%_net_gen_waste']

In [31]:
# Create subtable for CAPEX and rename fields to match fuel type
ferc_cap = ferc_eia_pcts[['plant_id_pudl','report_year','capex_coal','capex_gas','capex_oil','capex_waste']]
ferc_cap = ferc_cap.rename(columns={'capex_coal':'coal','capex_gas':'gas','capex_oil':'oil','capex_waste':'waste'})

# Melt by plant id and year and rename columns to be more descriptive
ferc_cap_melt = pd.melt(ferc_cap,['plant_id_pudl','report_year']).rename(columns={'value':'capex','variable':'fuel_type_code_pudl'})

# Delete all rows where the 'value' column is NaN (means there is no gas or oil for example)
ferc_cap_melt = ferc_cap_melt.dropna(subset=['capex'])

In [32]:
# Create subtable for OPEX and rename fields to match fuel type
ferc_op = ferc_eia_pcts[['plant_id_pudl','report_year','opex_coal','opex_gas','opex_oil','opex_waste']]
ferc_op = ferc_cap.rename(columns={'opex_coal':'coal','opex_gas':'gas','opex_oil':'oil','opex_waste':'waste'})

# Melt by plant id and year and rename columns to be more descriptive
ferc_op_melt = pd.melt(ferc_op,['plant_id_pudl','report_year']).rename(columns={'value':'opex','variable':'fuel_type_code_pudl'})

# Delete all rows where the 'value' column is NaN (means there is no gas or oil for example)
ferc_op_melt = ferc_op_melt.dropna(subset=['opex'])

In [33]:
# --- SANITY CHECK --- 
#ferc_cap_melt.loc[ferc_cap_melt['plant_id_pudl']==32].sort_values('report_year')
#ferc_cap.loc[ferc_cap['plant_id_pudl']==32].sort_values('report_year')

test = ferc_cap.groupby(['plant_id_pudl','report_year']).count()
test['count'] = test[['coal','gas','oil','waste']].max(axis=1)
no_eia_values = len(test.loc[test['count']==0])
ferc_len = len(ferc_cap)
missing = ferc_len-no_eia_values
ferc_melt_len = len(ferc_cap_melt)
#print(missing,ferc_melt_len) # why are these different?

#ferc_op_melt.loc[ferc_op_melt['plant_id_pudl']==32].sort_values('report_year')
#ferc_op.loc[ferc_op['plant_id_pudl']==32].sort_values('report_year')

In [34]:
# Merge CAPEX and OPEX FERC tables 
ferc_cap_op = pd.merge(ferc_cap_melt, ferc_op_melt, on=['plant_id_pudl','fuel_type_code_pudl','report_year'], how='outer')

In [35]:
# --- SANITY CHECK ---
#ferc_cap_op.loc[ferc_cap_op['plant_id_pudl']==32].sort_values('report_year')
#len(ferc_cap_op)

##### **FERC & EIA Merge #2:** <font color=grey>*eia_ferc_merge_df*</font> 
Pull together newly disaggregated FERC data with previously disaggregated EIA data to prep for MCOE calculations by plant and fuel type

In [36]:
# Merge FERC and EIA on plant, fuel, and year
eia_ferc_merge = pd.merge(eia_plant_fuel_ag, ferc_cap_op, on=['plant_id_pudl','fuel_type_code_pudl','report_year'], how='outer')

# rename columns to specify where they are coming from
eia_ferc_merge = eia_ferc_merge.rename(columns={
    'total_fuel_cost':'total_fuel_cost_eia',
    'net_generation_mwh':'net_generation_mwh_eia',
    'capacity_mw':'capacity_mw_eia',
    'capex':'capex_ferc',
    'opex':'opex_ferc'
})

In [37]:
# --- SANITY CHECK --- 
#eia_ferc_merge.loc[eia_ferc_merge['plant_id_pudl']==32].sort_values('report_year')
#len(eia_ferc_merge)

In [38]:
# Function to calculate MCOE
def calc_mcoe(fuel_cost_mwh, var_om_mwh, fixed_om, mw_capacity, net_mwh_gen):
    if net_mwh_gen > 0:
        mcoe = ((fuel_cost_mwh + var_om_mwh) + fixed_om * mw_capacity) / net_mwh_gen
    else:
        mcoe = None
    return mcoe

In [39]:
# Create new column with MCOE value
eia_ferc_merge['mcoe'] = eia_ferc_merge.apply(lambda x: calc_mcoe(x['total_fuel_cost_eia'],
                                                                  x['opex_ferc'],
                                                                  x['capex_ferc'],
                                                                  x['capacity_mw_eia'],
                                                                  x['net_generation_mwh_eia']),axis=1)

# Rearrange columns
eia_ferc_merge_df = eia_ferc_merge[[
    'plant_id_pudl',
    'fuel_type_code_pudl',
    'report_year',
    'total_fuel_cost_eia',
    'net_generation_mwh_eia',
    'capacity_mw_eia',
    'capex_ferc',
    'opex_ferc',
    'mcoe'
]]

In [50]:
# Change / add columns to match those desired in EI contract
mcoe_plant_df = eia_ferc_merge_df
mcoe_plant_df['fuel_cost_per_mwh'] = mcoe_plant_df['total_fuel_cost_eia'] / mcoe_plant_df['net_generation_mwh_eia']
mcoe_plant_df['variable_o&m_per_mwh'] = mcoe_plant_df['opex_ferc'] / mcoe_plant_df['net_generation_mwh_eia']
mcoe_plant_df['fixed_o&m_per_mwh'] = mcoe_plant_df['capex_ferc'] / mcoe_plant_df['net_generation_mwh_eia']
mcoe_plant_df = mcoe_plant_df.drop(['total_fuel_cost_eia','net_generation_mwh_eia','capacity_mw_eia','capex_ferc','opex_ferc'],axis=1)

<ipython-input-50-7f128174bf72>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mcoe_plant_df['fuel_cost_per_mwh'] = mcoe_plant_df['total_fuel_cost_eia'] / mcoe_plant_df['net_generation_mwh_eia']
<ipython-input-50-7f128174bf72>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mcoe_plant_df['variable_o&m_per_mwh'] = mcoe_plant_df['opex_ferc'] / mcoe_plant_df['net_generation_mwh_eia']
<ipython-input-50-7f128174bf72>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [52]:
mcoe_plant_df

,plant_id_pudl,fuel_type_code_pudl,report_year,mcoe,fuel_cost_per_mwh,variable_o&m_per_mwh,fixed_o&m_per_mwh
0,1,coal,2011,85636.821066,27.629311,118.621576,118.621576
1,1,coal,2012,87403.691577,34.294148,121.060548,121.060548
2,1,coal,2013,133391.912315,39.026661,184.776064,184.776064
3,1,coal,2015,120387.055479,44.946805,166.748107,166.748107
4,1,coal,2016,153179.563810,40.938561,212.191527,212.191527
...,...,...,...,...,...,...,...
3056,8446,gas,2017,NaN,NaN,NaN,NaN
3057,8446,gas,2018,NaN,NaN,NaN,NaN
3058,9938,gas,2017,137666.801594,12.320608,274.759443,274.759443
3059,9938,gas,2018,88795.670310,9.202056,177.218500,177.218500


In [ ]:
# --- SANITY CHECK --- 
#len(mcoe_plant_df)
#eia_ferc_merge.loc[eia_ferc_merge['plant_id_pudl']==32].sort_values('report_year')

##### **Plants With Significantly Different Hear Rates** <font color=grey>*test*</font> 
Using unit-level data from Part 1, find out whether there is any major discrepancies in heat rate for a given plant.

In [64]:
# Sum generator heat rate by plant and fuel type
plant_heat_rate = unit_level_df_all_years.groupby([
    'plant_id_pudl','fuel_type_code_pudl','report_date'],as_index=False)[
        'weighted_ave_heat_rate_mwh'].sum().rename(columns={
            'weighted_ave_heat_rate_mwh':'plant_heat_rate'})

# Merge back with unit-level data
plant_hr_merge = pd.merge(unit_level_df_all_years, plant_heat_rate, on=['plant_id_pudl','fuel_type_code_pudl','report_date'], how='outer')

In [66]:
# Delete unnecessary columns
plant_hr_merge = plant_hr_merge[['plant_id_pudl','unit_id_pudl','fuel_type_code_pudl',
                                 'report_date','weighted_ave_heat_rate_mwh','plant_heat_rate']]

# Add new column for calculation of whether its significantly different or not

In [70]:
import datetime
plant_hr_merge.loc[(plant_hr_merge['plant_id_pudl']==32)&(plant_hr_merge['report_date']==datetime.datetime(2017,1,1))]

,plant_id_pudl,unit_id_pudl,fuel_type_code_pudl,report_date,weighted_ave_heat_rate_mwh,plant_heat_rate
181,32,1,gas,2017-01-01,27.233530,65.913281
182,32,2,gas,2017-01-01,24.748185,65.913281
183,32,6,gas,2017-01-01,7.013889,65.913281
184,32,7,gas,2017-01-01,6.917677,65.913281
194,32,4,coal,2017-01-01,10.865694,20.747343
195,32,5,coal,2017-01-01,9.881649,20.747343
